## Обучение полносвязной нейронной сети с одним и несколькими слоями 

Выполнено на основе фреймворка Keras на бэкэнде Theano

In [1]:
import numpy as np
import os
import cv2
from keras.models import Model
from keras.layers import Input, Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
symbols = ['e', 'o', 'n', 'u']
data = []
classes = []

#составляем матрицу с изображениями и вектор их классов
for symbol in symbols:
    for image in os.listdir(symbol):
        array = cv2.imread(symbol + '/' + image, cv2.IMREAD_GRAYSCALE)
        data.append(array)
        if symbol == 'e':
            classes.append(0)
        elif symbol == 'n':
            classes.append(1)
        elif symbol == 'u':
            classes.append(2)
        else:
            classes.append(3)

arr_classes = np.array(classes)
arr_data = np.array(data)


In [5]:
#гиперпараметры
batch_size = 16 #обучающих примеров за раз
num_epochs = 15 #раз итерация по обучающему множеству

num_train = 640 #0.8 * 800 
num_test = 160 #0.2 * 800

#разбиение по людям
#так как человека у нас всего 2 в команде, то возьмем разбиение 50/50
#num_train = 400
#num_test = 400


height, width, depth = 32, 32, 1 #параметры изображений
num_classes = 4 #количество классов

#разбиение 80/20
x_train, x_test, y_train, y_test = train_test_split(arr_data, arr_classes, test_size = 0.2)

#приводим данные к 1Д 
x_train = x_train.reshape(num_train, height * width)
x_test = x_test.reshape(num_test, height * width)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#нормализация данных до значений [0,1]
x_train /= 255
x_test /= 255

#one-hot encoding
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

## Полносвязная нейросеть с одним слоем

In [6]:
inp = Input(shape = (height * width,)) #входной вектор
out = Dense(num_classes, activation='softmax')(inp) #выходной слой

model = Model(inputs=inp, outputs=out)

In [7]:
#accuracy 
model.compile(loss='categorical_crossentropy',#функция кросс-энтропии
             optimizer='adam', #оптимайзер Адама
             metrics=['accuracy']) 

In [8]:
model.summary()
#запуск обучающего алгоритма
model.fit(x_train, y_train, 
          batch_size=batch_size, epochs=num_epochs, 
         verbose=1, validation_split=0.1) #валидационный набор - 10%
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print('Loss = {}, Final accuracy = {}'.format(loss, accuracy))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 4100      
Total params: 4,100
Trainable params: 4,100
Non-trainable params: 0
_________________________________________________________________
Train on 576 samples, validate on 64 samples
Epoch 1/15
576/576 [==============================] - 15s 26ms/step - loss: 1.5442 - acc: 0.2587 - val_loss: 1.3553 - val_acc: 0.2188
Epoch 2/15
576/576 [==============================] - 16s 27ms/step - loss: 1.3690 - acc: 0.3212 - val_loss: 1.2334 - val_acc: 0.4375
Epoch 3/15
576/576 [==============================] - 16s 27ms/step - loss: 1.2946 - acc: 0.4010 - val_loss: 1.3161 - val_acc: 0.3281
Epoch 4/15
576/576 [==============================] - 15s 27ms/step - loss: 1.1822 - acc:

## Полносвязная нейросеть с 3 слоями

In [9]:
inp = Input(shape = (height * width,)) #входной вектор
hidden_1 = Dense(hidden_size, activation='relu')(inp) #первый слой с акт.ф-ей relu
hidden_2 = Dense(hidden_size, activation='relu')(hidden_1) #второй слой
out = Dense(num_classes, activation='softmax')(hidden_2) #выходной слой

model = Model(inputs=inp, outputs=out)

In [10]:
#accuracy 
model.compile(loss='categorical_crossentropy',#функция кросс-энтропии
             optimizer='adam', #оптимайзер Адама
             metrics=['accuracy']) 


In [ ]:
model.summary()
hidden_size = 10 #нейронов в двух слоях
#запуск обучающего алгоритма
model.fit(x_train, y_train, 
          batch_size=batch_size, epochs=num_epochs, 
         verbose=1, validation_split=0.1) #валидационный набор - 10%
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print('Loss = {}, Final accuracy = {}'.format(loss, accuracy))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               102500    
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 404       
Total params: 113,004
Trainable params: 113,004
Non-trainable params: 0
_________________________________________________________________
Train on 576 samples, validate on 64 samples
Epoch 1/15
576/576 [==============================] - 384s 667ms/step - loss: 1.5050 - acc: 0.2309 - val_loss: 1.4505 - val_acc: 0.1875
Epoch 2/15
576/576 [==============================] - 375s 652ms/step - loss: 1.3209 - 